In [4]:
# ---------- Principal Component Analysis (PCA) ----------

# ----- Step 1: Standardisation -----

# PCA is sensitive to variable variance, and so those those with larger ranges (e.g., between 0 and 100) will dominate over those with smaller ranges 
# (e.g., between 0 and 1). As such, standardisation is needed:

#                                                       z = (value - mean) / standard deviation



# ----- Step 2: Covariance Matrix Computation -----

# A covariance matrix is a p x p (where p = no. of dimensions) symmetric matrix that contains the covariances associated with all possible input 
# variable pairs, where the main diagonal contains the variances of each input variable. Correlated variables have a positive covariance, while
# inversely correlated variables have a negative covariance. A covariance matrix is thus nothing more than a table that summarises correlations 
# between all possible pairs.



# ----- Step 3: Compute Eigenvectors and Eigenvalues -----

# Eigenvectors (aka, the principal component; PC) indicate the direction in which the most variance occurs, while eigenvalues are coefficients that 
# quantify the amount of variance carried by each PC. Eigenvectors can be ranked in order of their eigenvalues, with the greatest eigenvalue
# corresponding to the first PC (i.e., PC1). The percentage of variance associated with a given component can be calculated by dividing its eigenvalue
# by the sum of eigenvalues.



# ----- Step 4: Create a Feature Vector -----

# A feature vector is a matrix containing the eigenvectors of features of interest. Components that carry a low percentage of overall variance may be
# excluded at this step, which will reduce dimensionality/information equivalent to that percentage. 



# ----- Step 5: Recast the Data Along the PC Axis -----

# In this step, we re-orient our data from the original axes to those represented by the PCs. This is done by multiplying the transpose of the 
# standardised dataset (SDS) acquired in step 1 by the transpose of the feature vector (FV) acquired in step 4:

# dataset = FV ^ T * SDS ^ T



# Source: https://builtin.com/data-science/step-step-explanation-principal-component-analysis

In [9]:
# ----- Import Packages -----

import numpy as np
from sklearn.decomposition import PCA

#from init import data
from numpy import cov
import numpy as np

from pkg import *



# ----- Step 1: Standardisation -----

def standardise(data):
    for col in data.columns:
        if col not in ['id', 'use', 'ir_agn', 'xray_agn', 'radio_agn', 'agn', 'true_agn']:
            data[col] = (data[col] - data[col].mean()) / data[col].std()
    return data

#datasets = data()
#cdfs, cos, uds = datasets['cdfs'], datasets['cos'], datasets['uds']

# Separate the columns to exclude from PCA
exclude_cols = ['id', 'use', 'ir_agn', 'xray_agn', 'radio_agn', 'agn', 'true_agn']

# Standardize each dataset, excluding certain columns
cdfs, cos, uds = cdfs.copy(), cos.copy(), uds.copy()

cdfs = standardise(cdfs.drop(columns=exclude_cols))
cos = standardise(cos.drop(columns=exclude_cols))
uds = standardise(uds.drop(columns=exclude_cols))

cdfs, cos, uds = standardise(cdfs), standardise(cos), standardise(uds)
cdfs, cos, uds = np.array(cdfs), np.array(cos), np.array(uds)
cdfs, cos, uds = np.nan_to_num(cdfs), np.nan_to_num(cos), np.nan_to_num(uds)



# ----- Step 2: Covariance Matrix Computation -----

pca_cdfs = PCA()
pca_cos = PCA()
pca_uds = PCA()

cdfs_sklearn = pca_cdfs.fit_transform(cdfs)
cos_sklearn = pca_cos.fit_transform(cos)
uds_sklearn = pca_uds.fit_transform(uds)

cov_matrix_cdfs, cov_matrix_cos, cov_matrix_uds = np.cov(cdfs, rowvar = False), np.cov(cos, rowvar = False), np.cov(uds, rowvar = False)



# ----- Step 3: Compute Eigenvectors and Eigenvalues -----

# Eigenvectors and eigenvalues are computed using NumPy's eig() function, which returns a tuple containing the eigenvectors and eigenvalues. The
# eigenvectors are stored in the columns of the matrix, and the eigenvalues are stored in a one-dimensional array.

eig_vals_cdfs, eig_vecs_cdfs = np.linalg.eig(cov_matrix_cdfs); eig_vals_cos, eig_vecs_cos = np.linalg.eig(cov_matrix_cos); eig_vals_uds, eig_vecs_uds = np.linalg.eig(cov_matrix_uds)

eig_pairs_cdfs, eig_pairs_cos, eig_pairs_uds = [(eig_vals_cdfs[i], eig_vecs_cdfs[:, i]) for i in range(len(eig_vals_cdfs))], [(eig_vals_cos[i], eig_vecs_cos[:, i]) for i in range(len(eig_vals_cos))], [(eig_vals_uds[i], eig_vecs_uds[:, i]) for i in range(len(eig_vals_uds))]

eig_pairs_cdfs.sort(key = lambda x: x[0], reverse = True); eig_pairs_cos.sort(key = lambda x: x[0], reverse = True); eig_pairs_uds.sort(key = lambda x: x[0], reverse = True)

sorted_eig_vecs_cdfs, sorted_eig_vecs_cos, sorted_eig_vecs_uds = np.array([eig_pairs_cdfs[i][1] for i in range(len(eig_vals_cdfs))]).T, np.array([eig_pairs_cos[i][1] for i in range(len(eig_vals_cos))]).T, np.array([eig_pairs_uds[i][1] for i in range(len(eig_vals_uds))]).T

# Align Signs

for i in range(len(eig_vecs_cdfs)):
    if np.dot(sorted_eig_vecs_cdfs[:, i], pca_cdfs.components_[i]) < 0:
        sorted_eig_vecs_cdfs[:, i] = -sorted_eig_vecs_cdfs[:, i]

for i in range(len(eig_vecs_cos)):
    if np.dot(sorted_eig_vecs_cos[:, i], pca_cos.components_[i]) < 0:
        sorted_eig_vecs_cos[:, i] = -sorted_eig_vecs_cos[:, i]

for i in range(len(eig_vecs_uds)):
    if np.dot(sorted_eig_vecs_uds[:, i], pca_uds.components_[i]) < 0:
        sorted_eig_vecs_uds[:, i] = -sorted_eig_vecs_uds[:, i]



# ----- Step 4: Create a Feature Vector -----
        
# The feature vector is created by sorting the eigenvectors based on their corresponding eigenvalues in descending order. The eigenvectors with the
# largest eigenvalues contain the most information about the data. The feature vector is a matrix where each column represents an eigenvector.

# Center Data
        
mean_cdfs, mean_cos, mean_uds = np.mean(cdfs, axis = 0), np.mean(cos, axis = 0), np.mean(uds, axis = 0)
centered_cdfs, centered_cos, centered_uds = cdfs - mean_cdfs, cos - mean_cos, uds - mean_uds

# Project Data

cdfs_manual, cos_manual, uds_manual = np.dot(centered_cdfs, sorted_eig_vecs_cdfs), np.dot(centered_cos, sorted_eig_vecs_cos), np.dot(centered_uds, sorted_eig_vecs_uds)

# Remove Imaginary Component
                                                                                                                                     
cdfs_manual, cos_manual, uds_manual = np.real(cdfs_manual), np.real(cos_manual), np.real(uds_manual)

# Print and Compare Results

np.savetxt('cdfs_manual.txt', cdfs_manual, delimiter = ',', fmt = '%.11f')
np.savetxt('cdfs_auto.txt', cdfs_sklearn, delimiter = ',', fmt = '%.11f')

print("Manual CDFS PCA matches sklearn PCA: ", np.allclose(cdfs_manual, cdfs_sklearn))
print("Manual COS PCA matches sklearn PCA: ", np.allclose(cos_manual, cos_sklearn))
print("Manual UDS PCA matches sklearn PCA: ", np.allclose(uds_manual, uds_sklearn))


Manual CDFS PCA matches sklearn PCA:  True
Manual COS PCA matches sklearn PCA:  True
Manual UDS PCA matches sklearn PCA:  True
